# Лабораторная работа 7. Управление ресурсами в однопроцессорной системе с неоднородными заявками
Выполнил: Пакало Александр Сергеевич, студент РТ5-81Б

## Задание 1
В однопроцессорную систему случайным образом поступают на выполнение $m$
разных типов программ, отличающихся известной трудоемкостью
$Q_1, Q_2,\dots, Q_m$.
Входящий поток простейший с интенсивностью $\lambda$.

Представив данную систему как одноканальную СМО с неограниченной очередью,
вычислить среднее время обслуживания программ, считая длительность
обслуживания случайной величиной (теоретически и экспериментально).

Реализовать алгоритм SPT, выбирая из очереди заявки в соответствии с их
приоритетом по трудоемкости. Рассчитать среднее время обслуживания программ.
Сравнить полученные результаты.

Реализовать алгоритм RR при заданном кванте времени $q$. Оценить среднее время
обслуживания программ. Сравнить полученные результаты. Выяснить, как влияет
величина кванта на среднее время обслуживания программ.

In [1]:
Variant <- 5
set.seed(Variant)
m <- sample(c(6:20), 1)
lambda <- runif(1, 0.1, 2)
Q <- rexp(m, 0.3)
q <- sample(c(1:4), 1)
View(data.frame(m, q, lambda))
print(Q)

m,q,lambda
<int>,<int>,<dbl>
7,1,1.401915


[1] 0.2417931 1.3403831 0.1863999 2.0529013 0.2552007 2.9172096 6.6709868


Заведем таблицу результатов

In [53]:
results <- data.frame("-", "-", "-", "-")
colnames(results) <- c("M/M/1/infty theoretical", "M/M/1/infty practical", "SPT", "Round Robin")
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<chr>,<chr>,<chr>,<chr>
-,-,-,-


### СМО вида $М/М/1/\infty$
Представим данную систему как одноканальную СМО с неограниченной очередью.

#### Теоретически

In [3]:
t2 <- mean(Q)
mu <- 1 / t2
mu

[1] 0.5122623

In [4]:
y <- lambda / mu
y

[1] 2.736714

Так как $y > 1$, поменяем $\lambda$, чтобы в системе не образовывалась
бесконечная очередь.

In [48]:
lambda <- 0.31

In [49]:
t2 <- mean(Q)
mu <- 1 / t2
mu

[1] 0.5122623

In [50]:
y <- lambda / mu
y

[1] 0.6051587

In [54]:
results[1] <- 1 / mu / (1 - y)
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<dbl>,<chr>,<chr>,<chr>
4.944075,-,-,-


#### Численно

In [9]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

env <- simmer("SuperDuperSim")
env

Loading required package: simmer



simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }

In [10]:
programs <- trajectory("programs' path") %>%
    seize("server", amount = 1) %>%
    timeout(function() rexp(1, mu)) %>%
    release("server", amount = 1)

In [11]:
SIMULATION_TIME <- 10000

env %>%
    add_resource("server", 1) %>%
    add_generator("programs", programs, function() rexp(1, lambda)) %>%
    run(until = SIMULATION_TIME)

simmer environment: SuperDuperSim | now: 10000 | next: 10003.7234989748
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: programs | monitored: 1 | n_generated: 3029 }

In [12]:
arrivals <- env %>%
    get_mon_arrivals()
arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programs0,1.282143,4.177776,2.89563311,TRUE,1
programs1,1.770443,4.258100,0.08032395,TRUE,1
programs2,4.408824,11.193970,6.78514566,TRUE,1
programs3,7.054637,13.693605,2.49963501,TRUE,1
programs4,11.334202,14.437538,0.74393268,TRUE,1
programs5,13.441375,18.684576,4.24703859,TRUE,1
programs6,19.294948,19.813446,0.51849776,TRUE,1
programs7,21.252343,21.511088,0.25874508,TRUE,1
programs8,25.357193,27.734487,2.37729349,TRUE,1


In [55]:
results[2] <- mean(arrivals %>% with(end_time - start_time))
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<dbl>,<dbl>,<chr>,<chr>
4.944075,4.906583,-,-


### Алгоритм SPT

In [14]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

spt.env <- simmer("SuperDuperSptSim")
spt.env

simmer environment: SuperDuperSptSim | now: 0 | next: 
{ Monitor: in memory }

Добавим $m$ генераторов. Каждый будет иметь приоритет в зависимости от
скорости выполнения программы. Генератор, создающий программы с наибольшей
длительностью выполнения, будет иметь наименьший приоритет.

In [15]:
SIMULATION_TIME <- 100000

spt.env %>%
    add_resource("server", 1)

Q_sorted_decr <- sort(Q, decreasing = TRUE)

spt.programs_trajectory <- function(time_to_execute) {
    return(
        trajectory("programs' path") %>%
            seize("server", 1) %>%
            timeout(time_to_execute) %>%
            release("server", 1)
    )
}

for (Q_i in seq_along(Q_sorted_decr)) {
    priority <- Q_i

    name <- paste0("programs", Q_i)

    spt.env %>% add_generator(
        name,
        spt.programs_trajectory(Q_sorted_decr[Q_i]),
        priority = priority,
        preemptible = priority,
        distribution = function() rexp(1, lambda / m)
    )
}

spt.env %>%
    run(until = SIMULATION_TIME)

simmer environment: SuperDuperSptSim | now: 0 | next: 
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }

simmer environment: SuperDuperSptSim | now: 1e+05 | next: 100004.231080445
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: programs1 | monitored: 1 | n_generated: 4270 }
{ Source: programs2 | monitored: 1 | n_generated: 4238 }
{ Source: programs3 | monitored: 1 | n_generated: 4265 }
{ Source: programs4 | monitored: 1 | n_generated: 4219 }
{ Source: programs5 | monitored: 1 | n_generated: 4199 }
{ Source: programs6 | monitored: 1 | n_generated: 4267 }
{ Source: programs7 | monitored: 1 | n_generated: 4282 }

In [16]:
spt.arrivals <- spt.env %>%
    get_mon_arrivals()
spt.arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programs10,3.139379,9.810366,6.6709868,TRUE,1
programs70,5.394727,9.996766,0.1863999,TRUE,1
programs11,7.332261,16.667752,6.6709868,TRUE,1
programs50,11.933127,16.922953,0.2552007,TRUE,1
programs40,12.175471,18.263336,1.3403831,TRUE,1
programs30,13.050160,20.316237,2.0529013,TRUE,1
programs41,19.239142,21.656620,1.3403831,TRUE,1
programs31,19.806402,23.709522,2.0529013,TRUE,1
programs20,22.848134,26.626731,2.9172096,TRUE,1


In [56]:
results[3] <- mean(spt.arrivals %>% with(end_time - start_time))
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<dbl>,<dbl>,<dbl>,<chr>
4.944075,4.906583,3.865118,-


### Алгоритм Round Robin
Реализуем Round Robin с помощью simmer. Для этого воспользуемся механизмом
`leave` и `handle_unfinished`.
Алгоритм реализации метода Round Robin:

1. Устанавливаем каждой программе, появившейся в системе, атрибут
`execution_time` $\in Q$. В последующем этот атрибут будет обновляться, храня
значение времени, которое осталось до выполнения программы.

2. Занимаем ресурс (сервер) на время $ = min(q, \text{execution_time})$.

3. Освобождаем ресурс (сервер).

4. Уменьшаем `execution_time` на `q`.

5. Если $\text{execution_time} <= 0$, программа выполнилась.

6. Если $\text{execution_time} > 0$, выводим программу из траектории с помощью
`leave`.

7. Перехватываем программу, покинувшую траекторию, с помощью
`handle_unfinished`, и отправляем её снова в эту же траекторию. Так, мы по факту
освобождаем ресурс для следующих программ, а текущую программу отправляем
в конец очереди, как это и нужно в Round Robin.

Выполняем пункты 2-7 для каждой программы. Замечу, что при переходе программы
в `handle_unfinished`, у нее будет обновленное значение `execution_time` (с
вычтенным квантом).
Следовательно, программа, проходя цикл, будет потихонечку исполняться.

In [18]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

if (!require("parallel")) {
    install.packages("parallel")
}
library(parallel)

Loading required package: parallel



Round Robin показывает очень колеблющиеся результаты в зависимости от порядка
заявок, приходящих в систему. Для чистоты эксперимента, воспользуемся пакетом parallel,
позволяющим запустить параллельно N симуляций, используя всю мощь механизма
`fork` операционных систем на базе Unix.

In [33]:
rr.simulate <- function(simulation_time, q) {
    return(function(i) {
        rr.env <- simmer("SuperDuperRoundRobinSim")

        rr.execute_program_for_quant <- trajectory() %>%
            seize("server", 1) %>%
            timeout(function() min(get_attribute(rr.env, "execution_time"), q)) %>%
            set_attribute("execution_time", -q, mod = "+") %>%
            release("server", 1) %>%
            leave(
                function() get_attribute(rr.env, "execution_time") > 0
            )

        rr.programs <- trajectory() %>%
            set_attribute("execution_time", function() sample(Q, 1)) %>%
            handle_unfinished(
                trajectory() %>%
                    log_(
                        function() paste0("preempteed with execution_time: ", get_attribute(rr.env, "execution_time")),
                        level = 1
                    ) %>%
                    join(rr.execute_program_for_quant)
            ) %>%
            join(rr.execute_program_for_quant)

        rr.env %>%
            add_resource("server", 1, preemptive = TRUE) %>%
            add_generator(
                "programs",
                rr.programs,
                priority = 1,
                preemptible = 1,
                restart = TRUE,
                distribution = function() rexp(1, lambda)
            ) %>%
            run(until = simulation_time) %>%
            wrap()
    })
}

Также запустим эти N симуляций для ряда значений разных $q$, чтобы посмотреть, как меняется
эффективность алгоритма в зависимости от значения кванта.

In [42]:
SIMULATION_TIME <- 50000
N <- 2

quants <- c(0.01 * q, 0.1 * q, q, 2 * q, 3 * q, 4 * q, 5 * q)

rr.results <- mclapply(quants, function(quant) {
    rr.envs <- mclapply(1:N, rr.simulate(SIMULATION_TIME, quant))

    rr.arrivals <- rr.envs %>%
        get_mon_arrivals()

    return(mean(rr.arrivals %>% with(end_time - start_time)))
})

In [43]:
unlist(rr.results)

[1] 4.654941 4.741071 4.795544 5.184292 4.993538 4.708045 4.951520

In [32]:
rr.experiments <- data.frame(quant = quants, result = unlist(rr.results))
rr.experiments

quant,result
<dbl>,<dbl>
0.01,4.636532
0.10,4.866035
1.00,5.357112
2.00,5.394939
3.00,4.695806
4.00,4.585076
5.00,5.065363


In [57]:
results[4] <- unlist(rr.results)[2]
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>
4.944075,4.906583,3.865118,4.741071


## Вывод по Round Robin для разных $q$
По полученным данным можно заметить, что система демонстрирует лучшие
значения при малых $q \le 1$, причем чем квант меньше, тем ближе результат
к SPT. При $q > 1$ система начинает стремиться к значениям, полученных при
симуляции обычной $M/M/1/\infty$, и колебаться около них.

Смею предположить, что данные также сильно зависят от вектора $Q$.
Чем больше среднее значение $Q$, тем "неприхотливее" будет система к величине
$q$, а значит критическое значение кванта, при котором система будет
становиться эффективнее, будет выше.

## Вывод
Как видно, система, выполненная с помощью алгоритма Round Robin оказалась
эффективнее обычной системы, а система, реализованная с алгоритмом SPT - самой
эффективной (с точки зрения среднего времени пребывания в системе).